In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import causalml
import preprocessing as pp
import statsmodels.api as sm
from causalml.inference.meta import BaseSRegressor, BaseTRegressor, BaseXRegressor, BaseRRegressor, BaseSClassifier, BaseTClassifier, BaseXClassifier, BaseRClassifier
from causalml.feature_selection.filters import FilterSelect
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from copy import deepcopy
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import plot_roc_curve

In [ ]:
from DriftDetection import *

In [ ]:
#Use method on company data

In [ ]:
##Preprocessing

In [ ]:
def process_data(data):
    data = data.sort_values("LQ_FB_DT")

    # Transform treatment strings into binary
    #data.Recipe=data.Recipe.replace({'recipe_1':1,'recipe_2':2, "recipe_3":3})

    transformed_data=pp.transform_data_mvp(original_data=data, state_CR_df=[], export_x_vars=False)
    #Remove dates and premium variables
    remove = ["LQ_FB_PREM_AMT", "LQ_FB_MONTH", "LQ_FB_YEAR"]
    transformed_data.drop(remove, axis=1, inplace = True)
    X = transformed_data.drop(["BIND_CNT", "Recipe"], axis = 1)

    corr_mat = X.corr()
    upper = corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(np.bool))
    thr = 0.8
    freq_thr = 0
    to_drop = [column for column in upper.columns if any(upper[column] >= thr)]
    x_vars = list(corr_mat.abs().drop(to_drop, axis = 1).columns)

    for var in x_vars:
        if len(np.unique(X[var])) <= 2:
            #if X[var].mean() <= freq_thr:
            #    to_drop.append(var)
            if "State" in var:
                to_drop.append(var)
    transformed_data.drop(to_drop, axis = 1, inplace = True)
    #train_data, test_data=pp.split_transformed_data(transformed_data, split_rate=.7)
    return transformed_data, to_drop

In [ ]:
#Formatting the data right
data_loc = 's3://cortex-mit1008-lmdl-lmdata/sad/data/CONFIDENTIAL_INFORMATION_SAD_omega_two_3pd_data_25may2021_05-27-2021_16-13.csv'
data=pd.read_csv(data_loc)
data.Recipe=data.Recipe.replace({'control':0,'treatment':1})
may_data, to_drop = process_data(data)
may_data.drop(["BIND_CNT", "Recipe"],axis = 1, inplace = True)
may_data= may_data.sort_values("LQ_FB_DT")
may_dates = may_data["LQ_FB_DT"]
may_data.drop("LQ_FB_DT", axis = 1, inplace = True)

In [ ]:
#Output: Columns (17,45,72,172,177,178,179,180,181) have mixed types.Specify dtype option on import or set low_memory=False.
#Output: ['Recipe', 'LQ_FB_DT', 'LQ_FB_PREM_AMT', 'BIND_CNT', 'Num_Drivers', 'Num_Vehicles', 'pibsScore', 'Driver1_Age_cat', 'Min_Driver_Age_cat', 'Max_Driver_Age_cat', 'Veh1_price', 'Veh1_mileage', 'LQ_FB_YEAR', 'LQ_FB_MONTH', 'Veh1_Age', 'Veh1_Age_sPurchase', 'State_AZ', 'State_CO', 'State_CT', 'State_DE', 'State_FL', 'State_IA', 'State_IL', 'State_IN', 'State_KY', 'State_ME', 'State_MI', 'State_MN', 'State_MO', 'State_NC', 'State_NH', 'State_NM', 'State_NY', 'State_OH', 'State_OK', 'State_OR', 'State_PA', 'State_SC', 'State_TN', 'State_TX', 'State_UT', 'State_VA', 'State_WI', 'Lead_agg', 'Lead_email', 'Lead_other', 'Lead_ps', 'Occu_HOMEMAKER', 'Occu_MILITARY OTHER', 'Occu_RETIRED', 'Occu_STUDENT (COLLEGE JUNIOR/SENIOR)', 'Occu_UNEMPLOYED', 'residenceType_HOMEOWNER', 'residenceType_LIVES_WITH_PARENTS_OTHER', 'residenceType_MOBILE_HOME_OWNER', 'residenceType_RENTER', 'Edu_BACHELORS', 'Edu_HIGH_SCHOOL_OR_EQUIVALENT', 'Edu_MASTERS', 'Edu_OTHER', 'Edu_VOCATIONAL', 'Driver1_gender_MALE', 'Prior_BI_Limits_COMPANY_CAR_ONLY', 'Prior_BI_Limits_DRIVING_WITHOUT_INSURANCE', 'Prior_BI_Limits_High', 'Prior_BI_Limits_Low', 'Prior_BI_Limits_MILITARY_SERVICE', 'Prior_BI_Limits_Mid-High', 'Prior_BI_Limits_Mid-Low', 'Prior_BI_Limits_NEWLY_LICENSED', 'Prior_BI_Limits_NO_NEED', 'Prior_BI_Limits_stateMinimum', 'Veh1_Ownership_LEASED', 'Veh1_Ownership_OWNED', 'Veh1NewUsed_Used']

In [ ]:
kwargs = {"n": 10000, "shift": 50, "n_bootstraps": 500,
         "alpha": 0.95, "gamma": 0.05, "delta": 0.01, "min_samples": 100}
#aug_dates=aug_data["LQ_FB_DT"]
#aug_data.drop("LQ_FB_DT", axis = 1, inplace = True)
res = sliding_adjoint_window(may_data,may_dates, **kwargs)

In [ ]:
#Output: Plot